In [8]:
from ultralytics import YOLO
from ultralytics.utils.metrics import mask_iou
from torch import bitwise_or, zeros, Tensor
import cv2
import numpy as np
import pandas as pd

model = YOLO('runs/segment/train5 - color region/weights/best.pt')

# 2017_01_24__23_00_04_84__SDO_AIA_AIA_193
# 2017_01_20__23_00_04_84__SDO_AIA_AIA_193
# 2017_01_28__22_59_52_84__SDO_AIA_AIA_193
results = model('./data/val(region)/images/2017_01_28__22_59_52_84__SDO_AIA_AIA_193.png')

def read_and_normalize_mask(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    image = cv2.normalize(image, None, 0, 1, cv2.NORM_MINMAX)

    _, mask = cv2.threshold(image, 0.5, 1, cv2.THRESH_BINARY)

    mask = mask.reshape(-1, int(image.shape[1] * image.shape[0]))

    return mask

true_mask = Tensor(read_and_normalize_mask("./data/val(region)/masks/2017_01_28__22_59_52_84__SDO_AIA_AIA_193.png"))
for r in results:
    masks = r.masks.data.reshape(-1, 256*256).cpu()
    masks = np.any(np.array(masks), axis=0)
    masks = masks.reshape(-1, 256*256)
    masks = Tensor(masks)
    
    # print(masks.reshape(-1, 256*256))
    print(mask_iou(true_mask, masks))


image 1/1 /home/artipark/Studenti/tests/data/val(region)/images/2017_01_28__22_59_52_84__SDO_AIA_AIA_193.png: 256x256 2 coronal holes, 12.0ms
Speed: 0.3ms preprocess, 12.0ms inference, 0.7ms postprocess per image at shape (1, 3, 256, 256)


tensor([[0.8345]])
